In [350]:
import numpy as np
from scipy.stats import truncnorm, expon, norm, beta

#np.random.seed(seed = 666)

# goal: compute Prob(X > gamma) where X ~ exp(1)
gamma = 1

# sample size
n = 1000

# generate {x_i}_{i = 1}^n


In [19]:
def density(X):
    return expon.pdf(X, scale = 1)

def grad_density(X):
    return -expon.pdf(X, scale = 1)

def left_score_fn(X):
    Z = np.array([X])
    Y = np.transpose(Z)
    return np.divide(grad_density(Y), density(Y))
    
def right_score_fn(X):
    return np.transpose(left_score_fn(X))

def constr(X):
    return X - 0

def grad_constr(X):
    return np.ones(X.shape)

def k_mat(X, h, c):
    
    Z = np.array([X])
    Y = np.transpose(Z)
    term1 = Y - Z
    term2 = np.log(constr(Y)) - np.log(constr(Z))
    
    return np.exp(- np.square(term1) / 2 / h**2 - np.square(term2) * c / 2)

def grad_left_k_mat(X, h, c):
    
    Z = np.array([X])
    Y = np.transpose(Z)
    term1 = Y - Z
    term2 = np.log(constr(Y)) - np.log(constr(Z))
    term3 = np.divide(grad_constr(Y), constr(Y))
    
    return np.multiply(k_mat(X, h, c), - term1 / h**2 - c * np.multiply(term2, term3)), - term1 / h**2 - c * np.multiply(term2, term3)

def grad_right_k_mat(X, h, c):
    
    Z = np.array([X])
    Y = np.transpose(Z)
    term1 = Y - Z
    term2 = np.log(constr(Y)) - np.log(constr(Z))
    term3 = np.divide(grad_constr(Z), constr(Z))
    
    return np.multiply(k_mat(X, h, c), term1 / h**2 + c * np.multiply(term2, term3)), term1 / h**2 + c * np.multiply(term2, term3)
    
def Hessian_k_mat(X, h, c):
    
    Z = np.array([X])
    Y = np.transpose(Z)
    grad_left, byproduct_left = grad_left_k_mat(X, h, c)
    grad_right, byproduct_right = grad_right_k_mat(X, h, c)
    term = 1 / h**2 + c * np.multiply(np.divide(grad_constr(Y), constr(Y)), np.divide(grad_constr(Z), constr(Z)))
    
    return np.multiply(k_mat(X, h, c), term) + np.multiply(k_mat(X, h, c), np.multiply(byproduct_left, byproduct_right))

In [20]:
# X_2
h = 1
c = 1
X = expon.rvs(scale = 1, size = n)   # scale = 1 / \lambda
#X = truncnorm.rvs(0, np.Inf, loc = 0, scale = 0.5, size = n)   # choose scale = 1 or 0.5?
grad_left, _ = grad_left_k_mat(X, h, c)
grad_right, _ = grad_right_k_mat(X, h, c)
Kp = np.multiply(left_score_fn(X), np.multiply(k_mat(X, h, c), right_score_fn(X))) + np.multiply(left_score_fn(X), grad_right) +  \
     np.multiply(grad_left, right_score_fn(X)) + Hessian_k_mat(X, h, c)

def verify_Kp_exp(X, h, c):
    
    n = X.shape[0]
    Kp = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):
            x = X[i]
            y = X[j]
            t = np.square(x - y) / 2 / h**2
            tt = c * np.square(np.log(x) - np.log(y)) / 2
            ttt = ((x - y) / h**2 + c * (np.log(x) - np.log(y)) / y) * ((y - x) / h**2 + c * (np.log(y) - np.log(x)) / x)
            Kp[i, j] += np.exp(- t - tt) + (-1) * ((x - y) / h**2 + c * (np.log(x) - np.log(y)) / y) * np.exp(- t - tt) + (-1) * ((y - x) / h**2 + c * (np.log(y) - np.log(x)) / x) * np.exp(- t - tt) + np.exp(- t - tt) * (1 / h**2 + c / x / y) + np.exp(- t - tt) * ttt
    
    return Kp

Kp_v = verify_Kp_exp(X, h, c)
np.square((Kp_v - Kp)).sum()

In [21]:
from cvxopt import matrix, solvers

d = Kp.shape[0]
Q = 2 * matrix(Kp)
p = matrix(np.zeros(d))
G = matrix(- np.identity(d))
h = matrix(np.zeros(d))
A = matrix(np.ones(d), (1, d))
b = matrix(1.0)
res = solvers.qp(Q, p, G, h, A, b)
w = np.array(res['x'])

def hh(X, gamma):
    Y = np.array([X])
    return Y > gamma

     pcost       dcost       gap    pres   dres
 0:  1.3581e-01 -9.1304e-01  4e+03  7e+01  7e+01
 1:  1.4016e-01 -9.0155e-01  9e+01  1e+00  1e+00
 2:  1.6663e-01 -8.3845e-01  1e+01  1e-01  1e-01
 3:  2.0066e-01 -9.1518e-01  4e+00  4e-02  4e-02
 4:  2.1308e-01 -6.3299e-01  1e+00  9e-03  1e-02
 5:  2.0716e-01 -3.1487e-01  7e-01  4e-03  4e-03
 6:  1.8392e-01 -1.4689e-01  4e-01  2e-03  2e-03
 7:  1.5813e-01 -3.4226e-02  2e-01  2e-16  2e-12
 8:  1.4608e-01  1.1927e-01  3e-02  9e-16  2e-12
 9:  1.4554e-01  1.2781e-01  2e-02  8e-16  4e-12
10:  1.4506e-01  1.4182e-01  3e-03  9e-16  1e-12
11:  1.4497e-01  1.4358e-01  1e-03  4e-17  8e-13
12:  1.4494e-01  1.4424e-01  7e-04  7e-16  2e-12
13:  1.4492e-01  1.4468e-01  2e-04  7e-16  7e-13
14:  1.4492e-01  1.4474e-01  2e-04  6e-16  3e-12
15:  1.4491e-01  1.4483e-01  8e-05  2e-16  9e-13
16:  1.4491e-01  1.4486e-01  5e-05  2e-16  3e-12
17:  1.4491e-01  1.4489e-01  2e-05  1e-15  1e-12
18:  1.4491e-01  1.4490e-01  6e-06  1e-15  2e-12
19:  1.4491e-01  1.44

In [22]:
print('Theoretical min', Kp.sum() / n**2)  
print('Actual min', np.dot(np.transpose(w), np.dot(Kp, w)))

Theoretical min 19.42688166392626
Actual min [[0.14490394]]


In [25]:
# true answer
1 - expon.cdf(gamma, scale = 1)

0.36787944117144233

In [23]:
# Stein est
est = np.dot(hh(X, gamma), w)
est

array([[0.65145985]])

In [24]:
# MC est
hh(X, gamma).mean()

0.36575

In [26]:
# test c = 0

In [56]:
def density(X):
    return norm.pdf(X)

def grad_density(X):
    return np.multiply(-X, norm.pdf(X))

def left_score_fn(X):
    Z = np.array([X])
    Y = np.transpose(Z)
    return np.divide(grad_density(Y), density(Y))
    
def right_score_fn(X):
    return np.transpose(left_score_fn(X))

def k_mat(X, h):
    Z = np.array([X])
    Y = np.transpose(Z)
    term = Y - Z
    return np.exp(- np.square(term) / 2 / h**2)

def grad_left_k_mat(X, h):
    Z = np.array([X])
    Y = np.transpose(Z)
    term = (Y - Z) / h**2
    return np.multiply(k_mat(X, h), - term), - term

def grad_right_k_mat(X, h):
    grad_left, term = grad_left_k_mat(X, h)
    return np.transpose(grad_left), -term
    
def Hessian_k_mat(X, h):
    Z = np.array([X])
    Y = np.transpose(Z)
    grad_left, byproduct_left = grad_left_k_mat(X, h)
    grad_right, byproduct_right = grad_right_k_mat(X, h)
    return k_mat(X, h) / h**2 + np.multiply(k_mat(X, h), np.multiply(byproduct_left, byproduct_right))

def verify_Kp(X, h):
    
    n = X.shape[0]
    Kp = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):
            x = X[i]
            y = X[j]
            t = np.square(x - y) / 2 / h**2
            Kp[i, j] += x * y * np.exp(-t) + (-x) * (x - y) * np.exp(-t) / h**2 + (-y) * (y - x) * np.exp(-t) / h**2 + np.exp(-t) / h**2 - (x - y)**2 * np.exp(-t) / h**4

    return Kp

In [58]:
from cvxopt import matrix, solvers
solvers.options['show_progress'] = False

def hh(X, gamma):
    Y = np.array([X])
    return Y > gamma

def test_h(h, n = 1000, gamma = 1, iters = 100):
    
    est_mat = np.zeros(iters)
    MC_mat = np.zeros(iters)
    
    for i in range(iters):
        
        #print('i =', i)
        
        X = np.random.normal(0, 1, n)

        grad_left, _ = grad_left_k_mat(X, h)
        grad_right, _ = grad_right_k_mat(X, h)
        Kp = np.multiply(left_score_fn(X), np.multiply(k_mat(X, h), right_score_fn(X))) + np.multiply(left_score_fn(X), grad_right) +  \
             np.multiply(grad_left, right_score_fn(X)) + Hessian_k_mat(X, h)
        #Kp_v = verify_Kp(X, h)
        #Kp_v
        #(Kp_v - Kp).sum()
        
        #print('Kp sum', Kp.sum())
        #print('Theoretical min', Kp.sum() / n**2)
    
        d = Kp.shape[0]
        Q = 2 * matrix(Kp)
        p = matrix(np.zeros(d))
        G = matrix(- np.identity(d))
        H = matrix(np.zeros(d))
        A = matrix(np.ones(d), (1, d))
        b = matrix(1.0)
        res = solvers.qp(Q, p, G, H, A, b)
        w = np.array(res['x'])
        
        #print('Actual min', np.dot(np.transpose(w), np.dot(Kp, w)))
    
        est = np.dot(hh(X, gamma), w)
        est_mat[i] = est
        MC_mat[i] = hh(X, gamma).mean()
        
    return est_mat, MC_mat

In [36]:
# true answer
1 - norm.cdf(1)

0.15865525393145707

In [37]:
est_mat, MC_mat = test_h(h = 1)

# MC mean
MC_mat.mean()

0.15789

In [38]:
# MC MSE
np.square(MC_mat - (1 - norm.cdf(1))).sum() / MC_mat.shape[0]

0.0001434835135796105

In [39]:
# Stein mean
est_mat.mean()

0.1612923026954622

In [40]:
# Stein MSE
np.square(est_mat - (1 - norm.cdf(1))).sum() / est_mat.shape[0]

0.0002441128440925012

In [59]:
est_mat, MC_mat = test_h(h = 0.2)

# MC mean
MC_mat.mean()

0.15767

In [60]:
# MC MSE
np.square(MC_mat - (1 - norm.cdf(1))).sum() / MC_mat.shape[0]

0.00013315182530945164

In [61]:
# Stein mean
est_mat.mean()

0.15690506130205986

In [62]:
# Stein MSE
np.square(est_mat - (1 - norm.cdf(1))).sum() / est_mat.shape[0]

1.060220104988912e-05

In [45]:
# true answer
1 - norm.cdf(2.5)

0.006209665325776159

In [46]:
est_mat, MC_mat = test_h(h = 1, gamma = 2.5)

# MC mean
MC_mat.mean()

0.005900000000000001

In [47]:
# MC MSE
np.square(MC_mat - (1 - norm.cdf(2.5))).sum() / MC_mat.shape[0]

5.605892613988053e-06

In [48]:
# Stein mean
est_mat.mean()

0.005920983841653566

In [49]:
# Stein MSE
np.square(est_mat - (1 - norm.cdf(2.5))).sum() / est_mat.shape[0]

8.165402029830791e-06

In [63]:
# what if we feed exponential r.v.'s?

from cvxopt import matrix, solvers
solvers.options['show_progress'] = False

def hh(X, gamma):
    Y = np.array([X])
    return Y > gamma

def test_h(alpha, h, n = 1000, gamma = 1, iters = 100):
    
    est_mat = np.zeros(iters)
    MC_mat = np.zeros(iters)
    
    for i in range(iters):
        
        print('i =', i)
        
        X = np.concatenate((expon.rvs(size = int(n * alpha)), -expon.rvs(size = int(n * (1 - alpha)))))

        grad_left, _ = grad_left_k_mat(X, h)
        grad_right, _ = grad_right_k_mat(X, h)
        Kp = np.multiply(left_score_fn(X), np.multiply(k_mat(X, h), right_score_fn(X))) + np.multiply(left_score_fn(X), grad_right) +  \
             np.multiply(grad_left, right_score_fn(X)) + Hessian_k_mat(X, h)
        #Kp_v = verify_Kp(X, h)
        #Kp_v
        #(Kp_v - Kp).sum()
        print('Kp sum', Kp.sum())
        print('Theoretical min', Kp.sum() / n**2)
    
        d = Kp.shape[0]
        Q = 2 * matrix(Kp)
        p = matrix(np.zeros(d))
        G = matrix(- np.identity(d))
        H = matrix(np.zeros(d))
        A = matrix(np.ones(d), (1, d))
        b = matrix(1.0)
        res = solvers.qp(Q, p, G, H, A, b)
        w = np.array(res['x'])
        
        print('Actual min', np.dot(np.transpose(w), np.dot(Kp, w)))
    
        est = np.dot(hh(X, gamma), w)
        est_mat[i] = est
        MC_mat[i] = hh(X, gamma).mean()
        
    return est_mat, MC_mat

In [51]:
# true answer
1 - norm.cdf(1)

0.15865525393145707

In [8]:
est_mat, MC_mat = test_h(alpha = 1, h = 1)

# Stein mean
est_mat.mean()

i = 0
Kp sum 544532.8421706052
Theoretical min 0.5445328421706052
Actual min [[0.2615803]]
i = 1
Kp sum 528371.7554948394
Theoretical min 0.5283717554948394
Actual min [[0.26128838]]
i = 2
Kp sum 549015.9918330084
Theoretical min 0.5490159918330083
Actual min [[0.26158256]]
i = 3
Kp sum 597205.8561753645
Theoretical min 0.5972058561753645
Actual min [[0.26184885]]
i = 4
Kp sum 505840.3528251238
Theoretical min 0.5058403528251239
Actual min [[0.26150237]]
i = 5
Kp sum 540085.9850134862
Theoretical min 0.5400859850134863
Actual min [[0.26146455]]
i = 6
Kp sum 531902.1958725034
Theoretical min 0.5319021958725034
Actual min [[0.26169938]]
i = 7
Kp sum 545596.253049952
Theoretical min 0.545596253049952
Actual min [[0.26080598]]
i = 8
Kp sum 527798.5044122601
Theoretical min 0.5277985044122602
Actual min [[0.26146421]]
i = 9
Kp sum 538076.2167094966
Theoretical min 0.5380762167094966
Actual min [[0.2608464]]


0.39141661284402085

In [10]:
# Stein MSE
np.square(est_mat - (1 - norm.cdf(1))).sum() / est_mat.shape[0]

0.054178428676178905

In [52]:
est_mat, MC_mat = test_h(alpha = 0.95, h = 1)

# Stein mean
est_mat.mean()

i = 0
Kp sum 457456.51841712877
Theoretical min 0.45745651841712875
Actual min [[2.70298427e-09]]
i = 1
Kp sum 426998.81489222107
Theoretical min 0.4269988148922211
Actual min [[1.57484713e-08]]
i = 2
Kp sum 446794.6978555394
Theoretical min 0.4467946978555394
Actual min [[9.3267518e-09]]
i = 3
Kp sum 450946.52161277406
Theoretical min 0.45094652161277404
Actual min [[1.53219609e-09]]
i = 4
Kp sum 419912.3932520617
Theoretical min 0.4199123932520617
Actual min [[2.61465898e-05]]
i = 5
Kp sum 437055.027564296
Theoretical min 0.437055027564296
Actual min [[2.20691188e-09]]
i = 6
Kp sum 425432.4937686699
Theoretical min 0.4254324937686699
Actual min [[1.4234194e-08]]
i = 7
Kp sum 421113.24450768525
Theoretical min 0.4211132445076852
Actual min [[2.84217634e-08]]
i = 8
Kp sum 448838.90737054544
Theoretical min 0.44883890737054544
Actual min [[1.83274008e-08]]
i = 9
Kp sum 443666.37655419466
Theoretical min 0.44366637655419466
Actual min [[4.53040777e-09]]
i = 10
Kp sum 429651.8858213902
Th

Actual min [[1.20737416e-06]]
i = 85
Kp sum 421488.2816581543
Theoretical min 0.4214882816581543
Actual min [[5.47673551e-08]]
i = 86
Kp sum 436449.97036146844
Theoretical min 0.43644997036146843
Actual min [[2.96976709e-05]]
i = 87
Kp sum 443141.7080805069
Theoretical min 0.4431417080805069
Actual min [[6.38964391e-09]]
i = 88
Kp sum 423056.2670641882
Theoretical min 0.4230562670641882
Actual min [[3.78252289e-08]]
i = 89
Kp sum 450317.75908066414
Theoretical min 0.4503177590806641
Actual min [[4.06583715e-10]]
i = 90
Kp sum 445201.7674750313
Theoretical min 0.4452017674750313
Actual min [[3.31504726e-09]]
i = 91
Kp sum 448711.96745436394
Theoretical min 0.4487119674543639
Actual min [[4.96857941e-08]]
i = 92
Kp sum 443470.05478926795
Theoretical min 0.44347005478926793
Actual min [[3.37420963e-09]]
i = 93
Kp sum 426667.50318171526
Theoretical min 0.42666750318171526
Actual min [[3.874365e-09]]
i = 94
Kp sum 452360.3833899954
Theoretical min 0.4523603833899954
Actual min [[7.30759837e

0.1546732914198004

In [53]:
# Stein MSE
np.square(est_mat - (1 - norm.cdf(1))).sum() / est_mat.shape[0]

0.00020415042238649893

In [64]:
est_mat, MC_mat = test_h(alpha = 0.75, h = 0.2)

# Stein mean
est_mat.mean()

i = 0
Kp sum 287665.1430983296
Theoretical min 0.2876651430983296
Actual min [[9.75906677e-07]]
i = 1
Kp sum 246516.52276857832
Theoretical min 0.24651652276857833
Actual min [[3.70533339e-05]]
i = 2
Kp sum 233764.87862367358
Theoretical min 0.23376487862367357
Actual min [[2.60101385e-07]]
i = 3
Kp sum 278916.5192690522
Theoretical min 0.2789165192690522
Actual min [[5.08872348e-08]]
i = 4
Kp sum 156696.66489663965
Theoretical min 0.15669666489663964
Actual min [[2.80344318e-07]]
i = 5
Kp sum 274770.49789027905
Theoretical min 0.274770497890279
Actual min [[7.90217029e-06]]
i = 6
Kp sum 288881.2038680939
Theoretical min 0.2888812038680939
Actual min [[1.00155452e-07]]
i = 7
Kp sum 228194.2428877319
Theoretical min 0.2281942428877319
Actual min [[6.50117862e-07]]
i = 8
Kp sum 240267.30592942124
Theoretical min 0.24026730592942125
Actual min [[9.94612606e-08]]
i = 9
Kp sum 313530.20801953884
Theoretical min 0.31353020801953885
Actual min [[6.80605822e-06]]
i = 10
Kp sum 166113.964725918

Actual min [[4.3957072e-07]]
i = 84
Kp sum 233557.6964554998
Theoretical min 0.23355769645549979
Actual min [[1.85905234e-08]]
i = 85
Kp sum 233094.21936947084
Theoretical min 0.23309421936947083
Actual min [[1.11119319e-05]]
i = 86
Kp sum 228512.1886053523
Theoretical min 0.2285121886053523
Actual min [[5.74976733e-06]]
i = 87
Kp sum 214879.55056357122
Theoretical min 0.21487955056357122
Actual min [[2.78993012e-06]]
i = 88
Kp sum 245026.39035128054
Theoretical min 0.24502639035128054
Actual min [[2.46999553e-08]]
i = 89
Kp sum 236067.0087514528
Theoretical min 0.2360670087514528
Actual min [[7.94319667e-07]]
i = 90
Kp sum 169776.76066780314
Theoretical min 0.16977676066780314
Actual min [[2.82289597e-06]]
i = 91
Kp sum 166209.87109554996
Theoretical min 0.16620987109554997
Actual min [[5.11587561e-06]]
i = 92
Kp sum 212225.3734589763
Theoretical min 0.2122253734589763
Actual min [[2.64801692e-07]]
i = 93
Kp sum 193783.36810266375
Theoretical min 0.19378336810266375
Actual min [[8.919

0.15894532485617152

In [65]:
# Stein MSE
np.square(est_mat - (1 - norm.cdf(1))).sum() / est_mat.shape[0]

1.8138913741225364e-06

In [239]:
# goal: compute Prob(X > gamma) where X ~ truncated normal(0, 1; [0, infinity))

In [330]:
def density(X):
    return truncnorm.pdf(X, 0, np.Inf, loc = 0, scale = 1)

def grad_density(X):
    return np.multiply(-X, truncnorm.pdf(X, 0, np.Inf, loc = 0, scale = 1))

def left_score_fn(X):
    Z = np.array([X])
    Y = np.transpose(Z)
    return np.divide(grad_density(Y), density(Y))
    
def right_score_fn(X):
    return np.transpose(left_score_fn(X))

def constr(X):
    return X - 0

def grad_constr(X):
    return np.ones(X.shape)

def k_mat(X, h, c):
    
    Z = np.array([X])
    Y = np.transpose(Z)
    term1 = Y - Z
    term2 = np.log(constr(Y)) - np.log(constr(Z))
    
    return np.exp(- np.square(term1) / 2 / h**2 - np.square(term2) * c / 2)

def grad_left_k_mat(X, h, c):
    
    Z = np.array([X])
    Y = np.transpose(Z)
    term1 = Y - Z
    term2 = np.log(constr(Y)) - np.log(constr(Z))
    term3 = np.divide(grad_constr(Y), constr(Y))
    
    return np.multiply(k_mat(X, h, c), - term1 / h**2 - c * np.multiply(term2, term3)), - term1 / h**2 - c * np.multiply(term2, term3)

def grad_right_k_mat(X, h, c):
    
    Z = np.array([X])
    Y = np.transpose(Z)
    term1 = Y - Z
    term2 = np.log(constr(Y)) - np.log(constr(Z))
    term3 = np.divide(grad_constr(Z), constr(Z))
    
    return np.multiply(k_mat(X, h, c), term1 / h**2 + c * np.multiply(term2, term3)), term1 / h**2 + c * np.multiply(term2, term3)
    
def Hessian_k_mat(X, h, c):
    
    Z = np.array([X])
    Y = np.transpose(Z)
    grad_left, byproduct_left = grad_left_k_mat(X, h, c)
    grad_right, byproduct_right = grad_right_k_mat(X, h, c)
    term = 1 / h**2 + c * np.multiply(np.divide(grad_constr(Y), constr(Y)), np.divide(grad_constr(Z), constr(Z)))
    
    return np.multiply(k_mat(X, h, c), term) + np.multiply(k_mat(X, h, c), np.multiply(byproduct_left, byproduct_right))

In [331]:
from cvxopt import matrix, solvers
solvers.options['show_progress'] = False

def hh(X, gamma):
    Y = np.array([X])
    return Y > gamma

def test_h(h, c, n = 1000, gamma = 1, iters = 100, mode = 1):
    
    est_mat = np.zeros(iters)
    MC_mat = np.zeros(iters)
    
    for i in range(iters):
        
        if mode == 1:
            X = truncnorm.rvs(0, np.Inf, loc = 0, scale = 1, size = n)
        elif mode == 2:
            X = expon.rvs(size = n)
        else:
            X = truncnorm.rvs(0, np.Inf, loc = 0, scale = 1, size = n) / 2

        grad_left, _ = grad_left_k_mat(X, h, c)
        grad_right, _ = grad_right_k_mat(X, h, c)
        Kp = np.multiply(left_score_fn(X), np.multiply(k_mat(X, h, c), right_score_fn(X))) + np.multiply(left_score_fn(X), grad_right) +  \
             np.multiply(grad_left, right_score_fn(X)) + Hessian_k_mat(X, h, c)
    
        d = Kp.shape[0]
        Q = 2 * matrix(Kp)
        p = matrix(np.zeros(d))
        G = matrix(- np.identity(d))
        H = matrix(np.zeros(d))
        A = matrix(np.ones(d), (1, d))
        b = matrix(1.0)
        res = solvers.qp(Q, p, G, H, A, b)
        w = np.array(res['x'])
    
        est = np.dot(hh(X, gamma), w)
        est_mat[i] = est
        MC_mat[i] = hh(X, gamma).mean()
        
    return est_mat, MC_mat

In [316]:
# true answer
ans = 1 - truncnorm.cdf(1, 0, np.Inf, loc = 0, scale = 1)
ans

0.31731050786291415

In [325]:
est_mat, MC_mat = test_h(h = 1, c = 1, n = 1000, iters = 100, mode = 1)

# MC mean
MC_mat.mean()

0.31854000000000005

In [326]:
# MC MSE
np.square(MC_mat - ans).sum() / MC_mat.shape[0]

0.00024622005091515616

In [327]:
# Stein mean
est_mat.mean()

0.39784657115136585

In [328]:
# Stein MSE
np.square(est_mat - ans).sum() / est_mat.shape[0]

0.009726073531384757

In [332]:
est_mat, MC_mat = test_h(h = 1, c = 1, n = 1000, iters = 100, mode = 2)

# Stein mean
est_mat.mean()

0.39215634542528205

In [333]:
# Stein MSE
np.square(est_mat - ans).sum() / est_mat.shape[0]

0.00921943782923424

In [334]:
est_mat, MC_mat = test_h(h = 1, c = 1, n = 1000, iters = 100, mode = 3)

# Stein mean
est_mat.mean()

0.29154407621830836

In [335]:
# Stein MSE
np.square(est_mat - ans).sum() / est_mat.shape[0]

0.009413472147940312

In [336]:
est_mat, MC_mat = test_h(h = 1, c = 1, n = 4000, iters = 100, mode = 1)

# Stein mean
est_mat.mean()

0.33782270415911314

In [337]:
# Stein MSE
np.square(est_mat - ans).sum() / est_mat.shape[0]

0.003873786665526392

In [338]:
est_mat, MC_mat = test_h(h = 1, c = 1, n = 4000, iters = 100, mode = 2)

# Stein mean
est_mat.mean()

0.38690024200056705

In [339]:
# Stein MSE
np.square(est_mat - ans).sum() / est_mat.shape[0]

0.00657514993460386

In [347]:
est_mat, MC_mat = test_h(h = 1, c = 1, n = 250, iters = 100, mode = 1)

# Stein mean
est_mat.mean()

0.3968438364108202

In [348]:
# Stein MSE
np.square(est_mat - ans).sum() / est_mat.shape[0]

0.014774097230503394

In [352]:
# goal: compute Prob((X - 0.5) * Indicator(X > 0.5)) where X ~ Beta(2, 2)

In [358]:
def density(X):
    return beta.pdf(X, a = 2, b = 2)

def grad_density(X):
    return 6 - 12 * X   # a = b = 2

def left_score_fn(X):
    Z = np.array([X])
    Y = np.transpose(Z)
    return np.divide(grad_density(Y), density(Y))
    
def right_score_fn(X):
    return np.transpose(left_score_fn(X))

def constr1(X):
    return X - 0

def grad_constr1(X):
    return np.ones(X.shape)

def constr2(X):
    return 1 - X

def grad_constr2(X):
    return -np.ones(X.shape)

def k_mat(X, h, c1, c2):
    
    Z = np.array([X])
    Y = np.transpose(Z)
    term1 = Y - Z
    term2 = np.log(constr1(Y)) - np.log(constr1(Z))
    term3 = np.log(constr2(Y)) - np.log(constr2(Z))
    
    return np.exp(- np.square(term1) / 2 / h**2 - np.square(term2) * c1 / 2 - np.square(term3) * c2 / 2)

def grad_left_k_mat(X, h, c1, c2):
    
    Z = np.array([X])
    Y = np.transpose(Z)
    term1 = Y - Z
    term2 = np.log(constr1(Y)) - np.log(constr1(Z))
    term3 = np.divide(grad_constr1(Y), constr1(Y))
    term4 = np.log(constr2(Y)) - np.log(constr2(Z))
    term5 = np.divide(grad_constr2(Y), constr2(Y))
    
    return np.multiply(k_mat(X, h, c1, c2), - term1 / h**2 - c1 * np.multiply(term2, term3) - c2 * np.multiply(term4, term5)), - term1 / h**2 - c1 * np.multiply(term2, term3) - c2 * np.multiply(term4, term5)

def grad_right_k_mat(X, h, c1, c2):
    
    Z = np.array([X])
    Y = np.transpose(Z)
    term1 = Y - Z
    term2 = np.log(constr1(Y)) - np.log(constr1(Z))
    term3 = np.divide(grad_constr1(Z), constr1(Z))
    term4 = np.log(constr2(Y)) - np.log(constr2(Z))
    term5 = np.divide(grad_constr2(Z), constr2(Z))
    
    return np.multiply(k_mat(X, h, c1, c2), term1 / h**2 + c1 * np.multiply(term2, term3) + c2 * np.multiply(term4, term5)), term1 / h**2 + c1 * np.multiply(term2, term3) + c2 * np.multiply(term4, term5)
    
def Hessian_k_mat(X, h, c1, c2):
    
    Z = np.array([X])
    Y = np.transpose(Z)
    grad_left, byproduct_left = grad_left_k_mat(X, h, c1, c2)
    grad_right, byproduct_right = grad_right_k_mat(X, h, c1, c2)
    term = 1 / h**2 + c1 * np.multiply(np.divide(grad_constr1(Y), constr1(Y)), np.divide(grad_constr1(Z), constr1(Z))) + c2 * np.multiply(np.divide(grad_constr2(Y), constr2(Y)), np.divide(grad_constr2(Z), constr2(Z)))
    
    return np.multiply(k_mat(X, h, c1, c2), term) + np.multiply(k_mat(X, h, c1, c2), np.multiply(byproduct_left, byproduct_right))

X = beta.rvs(a = 2, b = 2, size = 100)

h = 1
c1 = 1
c2 = 1
grad_left, _ = grad_left_k_mat(X, h, c1, c2)
grad_right, _ = grad_right_k_mat(X, h, c1, c2)
Kp = np.multiply(left_score_fn(X), np.multiply(k_mat(X, h, c1, c2), right_score_fn(X))) + np.multiply(left_score_fn(X), grad_right) +  \
    np.multiply(grad_left, right_score_fn(X)) + Hessian_k_mat(X, h, c1, c2)
Kp

def verify_Kp_beta(X, h, c1, c2):
    
    n = X.shape[0]
    Kp = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):
            x = X[i]
            y = X[j]
            term1 = - np.square(x - y) / 2 / h**2
            term2 = - c1 * np.square(np.log(x) - np.log(y)) / 2
            term3 = - c2 * np.square(np.log(1 - x) - np.log(1 - y)) / 2
            term4 = - ((x - y) / h**2 + c1 * (np.log(x) - np.log(y)) / x + c2 * (np.log(1 - x) - np.log(1 - y)) / (x - 1)) 
            term5 = (x - y) / h**2 + c1 * (np.log(x) - np.log(y)) / y + c2 * (np.log(1 - x) - np.log(1 - y)) / (y - 1)
            
            s_x = (1 - 2 * x) / x / (1 - x)
            s_y = (1 - 2 * y) / y / (1 - y)
            
            Kp[i, j] += np.exp(term1 + term2 + term3) * s_x * s_y + s_x * np.exp(term1 + term2 + term3) * term5 + s_y * np.exp(term1 + term2 + term3) * term4 + np.exp(term1 + term2 + term3) * ((1 / h**2 + c1 / x / y + c2 / (1 - x) / (1 - y)) + term4 * term5)
    
    return Kp

Kp_v = verify_Kp_beta(X, h = 1, c1 = 1, c2 = 1)
np.square((Kp_v - Kp)).sum()

In [367]:
from cvxopt import matrix, solvers
solvers.options['show_progress'] = False

def hh(X, gamma):
    Y = np.array([X])
    return np.multiply(Y > gamma, Y - gamma)

def test_h_c1_c2(h, c1, c2, n = 1000, gamma = 0.5, iters = 100):
    
    est_mat = np.zeros(iters)
    MC_mat = np.zeros(iters)
    
    for i in range(iters):
        
        X = beta.rvs(a = 2, b = 2, size = n)

        grad_left, _ = grad_left_k_mat(X, h, c1, c2)
        grad_right, _ = grad_right_k_mat(X, h, c1, c2)
        Kp = np.multiply(left_score_fn(X), np.multiply(k_mat(X, h, c1, c2), right_score_fn(X))) + np.multiply(left_score_fn(X), grad_right) +  \
             np.multiply(grad_left, right_score_fn(X)) + Hessian_k_mat(X, h, c1, c2)
    
        d = Kp.shape[0]
        Q = 2 * matrix(Kp)
        p = matrix(np.zeros(d))
        G = matrix(- np.identity(d))
        H = matrix(np.zeros(d))
        A = matrix(np.ones(d), (1, d))
        b = matrix(1.0)
        res = solvers.qp(Q, p, G, H, A, b)
        w = np.array(res['x'])
    
        est = np.dot(hh(X, gamma), w)
        est_mat[i] = est
        MC_mat[i] = hh(X, gamma).mean()
        
    return est_mat, MC_mat

In [368]:
# true answer by hand
ans = 3 / 32
ans

0.09375

In [374]:
est_mat, MC_mat = test_h_c1_c2(h = 1, c1 = 1, c2 = 1, n = 1000, gamma = 0.5, iters = 100)

# MC mean
MC_mat.mean()

0.0935252580816249

In [375]:
# MC MSE
np.square(MC_mat - ans).sum() / MC_mat.shape[0]

2.205809520118159e-05

In [376]:
# Stein mean
est_mat.mean()

0.0937197826593041

In [377]:
# Stein MSE
np.square(est_mat - ans).sum() / est_mat.shape[0]

1.0167164846223063e-05